In [1]:
%load_ext autoreload
%autoreload 3

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
import networkx as nx
import scipy
import sklearn
import time
import pickle
import warnings

# methods
from gaccord import GraphicalAccord, GraphicalConcord
from inverse_covariance import QuicGraphicalLasso
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri

# utils
import sys
sys.path.append('../utils')
from utils import standardize, partial_corr, partial_corr_to_precision, compute_average_norm, pseudo_BIC, gauss_BIC, proj_precision_mat
from generate_graphs import generate_erdos_renyi, generate_data

In [2]:
# select graph structure = ['hub_network', 'erdos_renyi']
graph_structure = 'erdos_renyi'

### Generate graph and data

In [3]:
if graph_structure == 'hub_network':
    # we use pre-made hub-network graph structure, which was constructed by the following procedure:
    # (1) create a Barabasi-Albert scale-free graph
    # (2) randomly choose 5% of the nodes to be hub nodes
    # (3) for each hub node, construct a complete sub-graph (clique)
    Skel = np.genfromtxt('../data/hub_network_structure.txt', delimiter=',')

    n, p = 500, 1000
    n_prop_to_p = [0.5]
    random_state = 2023
    lower_weight, upper_weight = 0.5, 1.0

    # projection method
    np.random.seed(random_state)
    edge_weights = np.random.uniform(low=lower_weight, high=upper_weight, size=(p,p))
    edge_signs = np.random.choice([-1,1], size=(p,p))
    Theta = np.multiply(edge_weights, edge_signs)
    Theta = np.multiply(Skel, Theta)
    Theta = np.tril(Theta) + np.tril(Theta).T
    nz_indx = np.nonzero(Theta)
    for i in range(100):
        Theta = proj_precision_mat(Theta, nz_indx)
        if np.linalg.cond(Theta) < 20:
            break

    Theta = np.real(Theta)
    # spread diagonal of precision matrix
    spread_diag=[1, 3]
    d = np.random.uniform(spread_diag[0], spread_diag[1], p)
    Theta = np.diag(d) @ Theta @ np.diag(d)
    Rho = partial_corr(Theta)
    Sigma = np.linalg.inv(Theta)

    Xs = generate_data(p, n_prop_to_p, Sigma, N=1, standardize=False, random_state=2023)
    X = Xs[0]

elif graph_structure == 'erdos_renyi':
    n, p = 500, 1000
    n_prop_to_p = [0.5]
    Theta, Sigma = generate_erdos_renyi(p, type='proj', edge_prob=0.01, lower_weight=0.5, upper_weight=1.0, spread_diag=[1, 3], random_state=2023)
    Rho = partial_corr(Theta)

    Xs = generate_data(p, n_prop_to_p, Sigma, N=1, standardize=False, random_state=2023)
    X = Xs[0]

### Run ACCORD

In [4]:
accord_pbics = []

S = np.matmul(X.T, X)/n
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.1 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 30)
if graph_structure == 'hub_network':
    lams_accord = lams[::-1][15:27]
elif graph_structure == 'erdos_renyi':
    lams_accord = lams[::-1][8:20]

random_state = 2023
np.random.seed(random_state)
for lam in lams_accord:
    # ACCORD
    model = GraphicalAccord(Omega_star=np.eye(p), lam1=lam, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
    model.fit(X)
    Omega_hat = model.omega_.toarray()
    Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
    
    accord_pbics.append(pseudo_BIC(X, Theta_hat, modified=False))

# use optimal lambda based on p-bic
best_lam_accord = lams_accord[np.argmin(accord_pbics)]
model = GraphicalAccord(Omega_star=np.eye(p), lam1=best_lam_accord, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Omega_hat = model.omega_.toarray()
Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
Rho_hat = partial_corr(Theta_hat)

# norm_total_accord, norm_TP_accord, norm_FP_accord, norm_FN_accord, norm_diag_accord, count_TP_accord, count_FP_accord, count_FN_accord = compute_average_norm(Theta, Theta_hat)
norm_total_accord, norm_TP_accord, norm_FP_accord, norm_FN_accord, norm_diag_accord, count_TP_accord, count_FP_accord, count_FN_accord = compute_average_norm(Rho, Rho_hat)

### Run ACCORD-SP

In [5]:
accord_sp_pbics = []

S = np.matmul(X.T, X)/n
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.1 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 30)
if graph_structure == 'hub_network':
    lams_accord_sp = lams[::-1][15:27]
elif graph_structure == 'erdos_renyi':
    lams_accord_sp = lams[::-1][8:20]

random_state = 2023
np.random.seed(random_state)
for lam in lams_accord_sp:
    # ACCORD
    model = GraphicalAccord(Omega_star=np.eye(p), lam1=lam, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
    model.fit(X)
    Omega_hat = model.omega_.toarray()

    # set w_ij = w_ji = 0 if at least one of them is 0 and average nonzero values
    zero_indices = np.where((Omega_hat == 0) | (Omega_hat.T == 0))
    Omega_hat[zero_indices] = 0
    Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
    
    accord_sp_pbics.append(pseudo_BIC(X, Theta_hat, modified=False))

# use optimal lambda based on p-bic
best_lam_accord = lams_accord[np.argmin(accord_sp_pbics)]
model = GraphicalAccord(Omega_star=np.eye(p), lam1=best_lam_accord, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Omega_hat = model.omega_.toarray()
zero_indices = np.where((Omega_hat == 0) | (Omega_hat.T == 0))
Omega_hat[zero_indices] = 0
Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
Rho_hat = partial_corr(Theta_hat)

# norm_total_accord_sp, norm_TP_accord_sp, norm_FP_accord_sp, norm_FN_accord_sp, norm_diag_accord_sp, count_TP_accord_sp, count_FP_accord_sp, count_FN_accord_sp = compute_average_norm(Theta, Theta_hat)
norm_total_accord_sp, norm_TP_accord_sp, norm_FP_accord_sp, norm_FN_accord_sp, norm_diag_accord_sp, count_TP_accord_sp, count_FP_accord_sp, count_FN_accord_sp = compute_average_norm(Rho, Rho_hat)

### Run CONCORD

In [6]:
concord_pbics = []

S = np.matmul(X.T, X)/n
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.1 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 30)
if graph_structure == 'hub_network':
    lams_concord = lams[::-1][15:27]
elif graph_structure == 'erdos_renyi':
    lams_concord = lams[::-1][8:20]

random_state = 2023
np.random.seed(random_state)
for lam in lams_concord:
    # CONCORD
    model = GraphicalConcord(Omega_star=np.eye(p), lam1=lam, backtracking=True, epstol=1e-7, maxitr=100)
    model.fit(X)
    Theta_hat = model.omega_.toarray()
    
    concord_pbics.append(pseudo_BIC(X, Theta_hat, modified=False))

# use optimal lambda based on p-bic
best_lam_concord = lams_concord[np.argmin(concord_pbics)]
model = GraphicalConcord(Omega_star=np.eye(p), lam1=best_lam_concord, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Theta_hat = model.omega_.toarray()
Rho_hat = partial_corr(Theta_hat)

# norm_total_concord, norm_TP_concord, norm_FP_concord, norm_FN_concord, norm_diag_concord, count_TP_concord, count_FP_concord, count_FN_concord = compute_average_norm(Theta, Theta_hat)
norm_total_concord, norm_TP_concord, norm_FP_concord, norm_FN_concord, norm_diag_concord, count_TP_concord, count_FP_concord, count_FN_concord = compute_average_norm(Rho, Rho_hat)

### Run Glasso

In [7]:
%%capture

glasso_gbics = []

S = np.matmul(X.T, X)/n
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.1 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 30)
if graph_structure == 'hub_network':
    lams_glasso = lams[::-1][15:27]
elif graph_structure == 'erdos_renyi':
    lams_glasso = lams[::-1][12:24]

random_state = 2023
np.random.seed(random_state)
for lam in lams_glasso:
    quic = QuicGraphicalLasso(lam=lam, max_iter=100, init_method='cov', auto_scale=False).fit(X)
    Theta_hat = quic.precision_

    glasso_gbics.append(gauss_BIC(X, Theta_hat))

# use optimal lambda based on g-bic
best_lam_glasso = lams_glasso[np.argmin(glasso_gbics)]
quic = QuicGraphicalLasso(lam=best_lam_glasso, max_iter=100, init_method='cov', auto_scale=False).fit(X)
Theta_hat = quic.precision_
Rho_hat = partial_corr(Theta_hat)

# norm_total_glasso, norm_TP_glasso, norm_FP_glasso, norm_FN_glasso, norm_diag_glasso, count_TP_glasso, count_FP_glasso, count_FN_glasso = compute_average_norm(Theta, Theta_hat)
norm_total_glasso, norm_TP_glasso, norm_FP_glasso, norm_FN_glasso, norm_diag_glasso, count_TP_glasso, count_FP_glasso, count_FN_glasso = compute_average_norm(Rho, Rho_hat)

### Run SPACE

In [8]:
%%capture

# import SPACE
rpy2.robjects.numpy2ri.activate()
space = importr('space')

space_pbics = []

if graph_structure == 'hub_network':
    lams_space = np.logspace(np.log10(40), np.log10(200), 12)
elif graph_structure == 'erdos_renyi':
    lams_space = np.logspace(np.log10(40), np.log10(200), 12)

random_state = 2023
np.random.seed(random_state)
for lam in lams_space:
    prec = space.space_joint(X, np.array([lam]))
    Theta_hat = np.array(prec[0])

    space_pbics.append(pseudo_BIC(X, Theta_hat, modified=False))

# use optimal lambda based on p-bic
best_lam_space = lams_space[np.argmin(space_pbics)]
prec = space.space_joint(X, np.array([best_lam_space]))
Theta_hat = partial_corr_to_precision(prec[0], prec[1])
Rho_hat = np.array(prec[0])

# norm_total_space, norm_TP_space, norm_FP_space, norm_FN_space, norm_diag_space, count_TP_space, count_FP_space, count_FN_space = compute_average_norm(Theta, Theta_hat)
norm_total_space, norm_TP_space, norm_FP_space, norm_FN_space, norm_diag_space, count_TP_space, count_FP_space, count_FN_space = compute_average_norm(Rho, Rho_hat)

In [9]:
print('ACCORD')
print(f'Total norm: {norm_total_accord:.2f}')
print(f'TP norm (# of TP): {norm_TP_accord:.3f} ({count_TP_accord})')
print(f'FP norm (# of FP): {norm_FP_accord:.3f} ({count_FP_accord})')
print(f'FN norm (# of FN): {norm_FN_accord:.3f} ({count_FN_accord})')
# print(f'Diag norm: {norm_diag_accord:.3f} ({p})', '\n')

print('ACCORD-SP')
print(f'Total norm: {norm_total_accord_sp:.2f}')
print(f'TP norm (# of TP): {norm_TP_accord_sp:.3f} ({count_TP_accord_sp})')
print(f'FP norm (# of FP): {norm_FP_accord_sp:.3f} ({count_FP_accord_sp})')
print(f'FN norm (# of FN): {norm_FN_accord_sp:.3f} ({count_FN_accord_sp})')
# print(f'Diag norm: {norm_diag_accord_sp:.3f} ({p})', '\n')

print('CONCORD')
print(f'Total norm: {norm_total_concord:.2f}')
print(f'TP norm (# of TP): {norm_TP_concord:.3f} ({count_TP_concord})')
print(f'FP norm (# of FP): {norm_FP_concord:.3f} ({count_FP_concord})')
print(f'FN norm (# of FN): {norm_FN_concord:.3f} ({count_FN_concord})')
# print(f'Diag norm: {norm_diag_concord:.3f} ({p})', '\n')

print('Glasso')
print(f'Total norm: {norm_total_glasso:.2f}')
print(f'TP norm (# of TP): {norm_TP_glasso:.3f} ({count_TP_glasso})')
print(f'FP norm (# of FP): {norm_FP_glasso:.3f} ({count_FP_glasso})')
print(f'FN norm (# of FN): {norm_FN_glasso:.3f} ({count_FN_glasso})')
# print(f'Diag norm: {norm_diag_glasso:.3f} ({p})', '\n')

print('SPACE')
print(f'Total norm: {norm_total_space:.2f}')
print(f'TP norm (# of TP): {norm_TP_space:.3f} ({count_TP_space})')
print(f'FP norm (# of FP): {norm_FP_space:.3f} ({count_FP_space})')
print(f'FN norm (# of FN): {norm_FN_space:.3f} ({count_FN_space})')
# print(f'Diag norm: {norm_diag_space:.3f} ({p})', '\n')

ACCORD
Total norm: 8.00
TP norm (# of TP): 0.110 (3458)
FP norm (# of FP): 0.012 (4685)
FN norm (# of FN): 0.123 (1439)
ACCORD-SP
Total norm: 7.60
TP norm (# of TP): 0.090 (2739)
FP norm (# of FP): 0.026 (1301)
FN norm (# of FN): 0.126 (2158)
CONCORD
Total norm: 7.31
TP norm (# of TP): 0.097 (3577)
FP norm (# of FP): 0.018 (3530)
FN norm (# of FN): 0.118 (1320)
Glasso
Total norm: 9.95
TP norm (# of TP): 0.123 (460)
FP norm (# of FP): 0.020 (79)
FN norm (# of FN): 0.144 (4437)
SPACE
Total norm: 10.16
TP norm (# of TP): 0.139 (129)
FP norm (# of FP): 0.016 (1)
FN norm (# of FN): 0.145 (4768)
